In [1]:
import mujoco
import mediapy as media
import numpy as np 
from robot_descriptions.a1_mj_description import MJCF_PATH
from scipy import interpolate
from utils import load_model_with_scene

In [2]:

model = load_model_with_scene(MJCF_PATH)
renderer = mujoco.Renderer(model, height=480, width=640)#, width=800, height=600)

In [3]:

data_file = 'run_twist_active.csv'

data_array = np.genfromtxt(data_file, delimiter=',', skip_header=100, skip_footer=100)
timespan = data_array[:, 0]- data_array[0, 0]
# sampling = np.mean(np.diff(timespan))
quaternion = data_array[:, 1:5]
joint_angles = data_array[:, 11:23] 
# linear_velocity = data_array[:, 5:8]
# joint_velocity = data_array[:, 23:35] 
# joint_torques = data_array[:, 35:47]
# contacts = data_array[:, 47:]
final_time = timespan[-1]


joint_angles_t = interpolate.interp1d(timespan, joint_angles, axis = 0)


In [ ]:
data = mujoco.MjData(model)
mujoco.mj_resetData(model, data)  # Reset state and time.
mujoco.mj_resetData(model, data)  # Reset state and time.
mujoco.mj_step(model, data)
model.opt.timestep = 1/1000

scene_option = mujoco.MjvOption()
scene_option.flags[mujoco.mjtVisFlag.mjVIS_CONTACTFORCE] = True
model.vis.map.force = 0.005

framerate = 30  # (Hz)

# Simulate and display video.
frames = []
data.qpos[0] = 0.
data.qpos[1] = 0.
data.qpos[2] = 0.31
data.qpos[3:7] = quaternion[0]
data.qpos[7:] = joint_angles[0]
data.qpos[7:] = joint_angles[0]
slow_down_factor = 1

while data.time <= 1.0*final_time:
    time = data.time
    q_joints = data.qpos[7:].copy()
    dq_joints = data.qvel[6:].copy()
    
    control = 120*(joint_angles_t(time) - q_joints) - 5*dq_joints
    data.ctrl = control #joint_angles_t(time )
    mujoco.mj_step(model, data)

    if len(frames) < slow_down_factor*time * framerate:
        renderer.update_scene(data, scene_option = scene_option)
        pixels = renderer.render()
        frames.append(pixels)
media.show_video(frames, fps=framerate)

